# COVID-19 Mortality Risk Analysis
## Research Questions
1️⃣ Which risk factors are more relevant or have more impact on the death of a person between January and June 2020?

2️⃣ Was COVID-19 the main cause of death during this period compared to other causes?

By: Miguel Trujillo Lopez

## Notebook Overview: Data Ingestion

This notebook covers the **data ingestion and preprocessing stage** of the COVID-19 mortality analysis project. The original dataset presents several real-world challenges, including large file size, malformed records, mixed data types, and non-standard encodings for missing values.

### Objectives
- Load a large-scale public health dataset under limited memory conditions.
- Resolve parsing issues caused by malformed CSV rows.
- Normalize data types to ensure analytical correctness.
- Explicitly encode patient mortality status.
- Persist a clean, analysis-ready dataset for downstream exploration.

### Main Steps Performed

**1. Data Ingestion**
- Loaded a dataset containing over 14 million patient-level records.
- Selected only analytically relevant columns to reduce memory usage.
- Used defensive loading strategies (`on_bad_lines`, column subsetting) to handle corrupted rows.

**2. Data Cleaning and Type Normalization**
- Converted all variables to string format during ingestion to avoid parsing failures.
- Standardized numerical clinical variables using safe numeric conversion.
- Parsed date fields and handled sentinel values such as `"9999-99-99"`.
- Optimized memory usage through nullable integer data types.

**3. Outcome Variable Creation**
- Derived a binary `DEATH` variable from the date-of-death field (`FECHA_DEF`).
- Encoded mortality explicitly to support risk-factor comparisons and statistical analysis.

**4. Data Persistence**
- Exported the cleaned dataset in Parquet format to preserve data types, reduce file size, and enable efficient reuse across notebooks.

This notebook concludes the **data preparation phase**. Subsequent notebooks build upon this cleaned dataset to perform feature engineering and exploratory data analysis.



### Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:.2f}".format)

sns.set_style("whitegrid")


### Importing Dataset

In [12]:
import re

input_path = "COVID19MEXICO2021.csv"
output_path = "COVID19MEXICO2021_clean.csv"

bad_lines = 0

with open(input_path, "r", encoding="utf-8", errors="replace") as fin, \
     open(output_path, "w", encoding="utf-8") as fout:

    for line in fin:
        fixed = re.sub(r'"(?=[^",]*,)|"(?=[^",]*$)', '', line)

        if fixed != line:
            bad_lines += 1

        fout.write(fixed)

print(f"Sanitization completed. Lines modified: {bad_lines}")


Sanitization completed. Lines modified: 11670850


In [2]:
import re

input_path = "COVID19MEXICO2021.csv"
output_path = "COVID19MEXICO2021_clean.csv"

fixed = 0
total = 0

with open(input_path, "r", encoding="utf-8", errors="replace") as fin, \
     open(output_path, "w", encoding="utf-8") as fout:
    for line in fin:
        total += 1
        if line.count('"') % 2 == 1:
            # Unbalanced quotes in this row: drop quotes to restore CSV structure
            line = line.replace('"', "")
            fixed += 1
        fout.write(line)

print(f"Done. Total lines: {total}, lines fixed (unbalanced quotes): {fixed}")


Done. Total lines: 14167461, lines fixed (unbalanced quotes): 1037


### Modify Datatypes from CColumns

In [10]:
usecols = [
    "SEXO","EDAD","TIPO_PACIENTE","NEUMONIA","INTUBADO","UCI",
    "DIABETES","EPOC","ASMA","INMUSUPR","HIPERTENSION",
    "CARDIOVASCULAR","OBESIDAD","RENAL_CRONICA","TABAQUISMO",
    "CLASIFICACION_FINAL","FECHA_DEF"
]

df = pd.read_csv(
    "COVID19MEXICO2021_clean.csv",
    usecols=usecols,
    dtype="string",
    engine="c",
    on_bad_lines="skip",
    low_memory=True
)

print(df.shape)
print(df.dtypes)


(14167460, 17)
SEXO                   string[python]
TIPO_PACIENTE          string[python]
FECHA_DEF              string[python]
INTUBADO               string[python]
NEUMONIA               string[python]
EDAD                   string[python]
DIABETES               string[python]
EPOC                   string[python]
ASMA                   string[python]
INMUSUPR               string[python]
HIPERTENSION           string[python]
CARDIOVASCULAR         string[python]
OBESIDAD               string[python]
RENAL_CRONICA          string[python]
TABAQUISMO             string[python]
CLASIFICACION_FINAL    string[python]
UCI                    string[python]
dtype: object


In [11]:
import pandas as pd

df["FECHA_DEF"] = df["FECHA_DEF"].replace("9999-99-99", pd.NA)
df["FECHA_DEF"] = pd.to_datetime(df["FECHA_DEF"], errors="coerce")


In [12]:
df["FECHA_DEF"].dtype
df["FECHA_DEF"].isna().value_counts()

,count
FECHA_DEF,
True,13859230
False,308230


In [13]:
code_cols = [
    "SEXO","EDAD","TIPO_PACIENTE","NEUMONIA","INTUBADO","UCI",
    "DIABETES","EPOC","ASMA","INMUSUPR","HIPERTENSION",
    "CARDIOVASCULAR","OBESIDAD","RENAL_CRONICA","TABAQUISMO",
    "CLASIFICACION_FINAL"
]

for c in code_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")


In [14]:
df[code_cols].dtypes


,0
SEXO,Float64
EDAD,Int64
TIPO_PACIENTE,Int64
NEUMONIA,Int64
INTUBADO,Int64
UCI,Int64
DIABETES,Int64
EPOC,Int64
ASMA,Int64
INMUSUPR,Int64


In [15]:
for c in code_cols:
    df[c] = df[c].astype("Int16")

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14167460 entries, 0 to 14167459
Data columns (total 17 columns):
 #   Column               Dtype         
---  ------               -----         
 0   SEXO                 Int16         
 1   TIPO_PACIENTE        Int16         
 2   FECHA_DEF            datetime64[ns]
 3   INTUBADO             Int16         
 4   NEUMONIA             Int16         
 5   EDAD                 Int16         
 6   DIABETES             Int16         
 7   EPOC                 Int16         
 8   ASMA                 Int16         
 9   INMUSUPR             Int16         
 10  HIPERTENSION         Int16         
 11  CARDIOVASCULAR       Int16         
 12  OBESIDAD             Int16         
 13  RENAL_CRONICA        Int16         
 14  TABAQUISMO           Int16         
 15  CLASIFICACION_FINAL  Int16         
 16  UCI                  Int16         
dtypes: Int16(16), datetime64[ns](1)
memory usage: 756.6 MB


### Creating a DEATH variable

In [18]:
df["DEATH"] = df["FECHA_DEF"].notna().astype("int8")

In [19]:
df["DEATH"].value_counts()

,count
DEATH,
0,13859230
1,308230


In [20]:
df.head()

,SEXO,TIPO_PACIENTE,FECHA_DEF,INTUBADO,NEUMONIA,EDAD,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,CLASIFICACION_FINAL,UCI,DEATH
0,2,1,NaT,97,2,26,2,2,2,2,2,2,2,2,2,7,97,0
1,1,1,NaT,97,99,34,2,2,2,2,2,2,2,2,2,7,97,0
2,2,1,NaT,97,2,41,2,2,2,2,2,2,2,2,1,7,97,0
3,2,1,NaT,97,2,25,2,2,2,2,2,2,2,2,2,7,97,0
4,1,1,NaT,97,2,20,2,2,2,2,2,2,2,2,2,7,97,0


### Save Dataset

In [22]:
df.to_csv("COVID19MEXICO2021_final.csv", index=False)

In [21]:
df.to_parquet("COVID19MEXICO2021_final.parquet", index=False)

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
!cp COVID19MEXICO2021_final.parquet "/content/drive/MyDrive/COVID19MEXICO2021_final.parquet"
!cp COVID19MEXICO2021_final.parquet "/content/drive/MyDrive/COVID19MEXICO2021_final.csv"


## Summary and Next Steps

This notebook focused on **data ingestion** for the COVID-19 2021 dataset. Given the size and quality challenges of the raw data, several preprocessing steps were required to ensure the dataset is reliable and analysis-ready.

Key outcomes of this stage include:
- Successful loading of a large-scale dataset (>14 million records) under memory constraints.
- Resolution of malformed CSV rows and inconsistent data encodings.
- Explicit normalization of data types for clinical and demographic variables.
- Creation of a binary outcome variable (`DEATH`) derived from the date-of-death field.
- Export of a clean, optimized dataset in Parquet format for efficient downstream analysis.

With these steps completed, the dataset is now suitable for exploratory data analysis.  
The next notebook will focus on **feature engineering and exploratory analysis**, aiming to identify the most relevant risk factors associated with mortality and to contextualize COVID-19 deaths relative to other outcomes during 2021.

